In [12]:
#Kuba Czech, 156035

# Informed search - the A* algorithm

The depth-first search and breadth-first search considered in the previous lesson are completely *blind* algorithms: they're only concerned whether the currently considered state is a goal state or not. They're unable to distinguish whether a state is easy or hard to reach, or whether it is near or far of the goal. This makes them very inefficient search algorithms. To allievate the issue, we introduce informed search algorithms. The information is given to an algorithm in two ways:

1. By using an *action cost* function $c(s,a)$, which, given a state $s$ and an action $a$ available in this state, returns its cost as a non-negative number.
2. By using a *heuristic* $h(s)$, which, given a state, estimates the lowest cost to reach a goal state from the given state.

Given a sequence of actions $a_1, \ldots, a_n$ and an initial state $s_1$, we can express the *total cost* of reaching the state $s_{n+1}$ by executing the sequence as:
$$ c(s_1, a_1, \ldots, a_{n-1}) = \sum_{i=1}^n c(s_i, a_i) $$
and the *expected cost* of the solution as the sum of the total cost and the estimate cost of reaching the goal from the state $s_{n+1}$
$$ f(s_1, a_1, \ldots, a_n) = c(s_1, a_1, \ldots, a_n) + h(s_{n+1}) $$

The heuristic function is a bit tricky, because we want it to have two properties:
* *Admissibility*: It must never *overestimate* the true cost of reaching the goal. 
* *Consistency*: Let $s$ be a state such that $a$ is an available action in this state and $s'$ is the state reached by executing this action. The heuristic should fulfil triangle inequality, that is, the estimated cost to reach the goal from $s$ should be no greater than the cost of executing the action $a$ + the estimated cost of reaching the goal from the new state.
$$ h(s) \leq c(s, a) + h(s') $$

One can prove that admissibility follows from consistency, but consistency is important only if there are multiple paths to reach the same state (i.e., we are searching in a graph, not in a tree). Otherwise, admissability is sufficient.

Lets extend the class `Problem` from the previous lesson with two new functions `action_cost` and `heuristic`, which correspond to the functions $c(s,a)$ and $h(s)$ described above.

In [1]:
class Problem:
    @property
    def initial_state(self):
        ...
        
    def available_actions(self, state):
        ...        
        
    def do_action(self, state, action):
        ...
        return new_state
    
    def is_goal(self, state) -> bool:
        ...
        
    def action_cost(self, state, action) -> float:
        ...
        
    def heuristic(self, state) -> float:
        ...

To make a concrete example, lets revisit the vacuum world. 

![](aima-images/fig2_2.png)

Below, we assume a very simple model:
* Any action costs 1. This corresponds to searching for the shortest plan.
* The heuristic estimation is the number of fields which are still dirty. 


Lets consider the properties of the heuristic:
* Is it admissible? The heuristic value is equal to the number of 'Suck' actions that are yet to be executed and ignores the spatial aspect (i.e., moving between the rooms), thus never overestimating.
* Is it consistent? As a consequence of a single action the heuristic value can decrease by at most 1 (if the action happens to be 'Suck' and the room is dirty). The cost of any action is 1, so rewriting the triangle inequality we arrive at:
$$ h(s) \leq c(s, a) + h(s') = \begin{cases} 1 + (h(s)-1) & a=\text{'Suck' and the room was dirty} \\ 1 + h(s) & \text{otherwise} \end{cases} $$
* Is it the best we could have? By no means! We could include the spatial aspect.

In [2]:
class VacuumProblem(Problem):
    @property
    def initial_state(self):
        return (0, (True, True))
    
    def available_actions(self, state):
        return ["Left", "Suck", "Right"]
        
    def do_action(self, state, action):
        robot, dirty = state
        if action == "Left":
            return (max(robot-1, 0), dirty)
        elif action == "Suck":
            new_dirty = list(dirty)
            new_dirty[robot] = False
            return (robot, tuple(new_dirty))
        elif action == "Right":
            return (min(robot+1, len(dirty)-1), dirty)        
        raise Exception('Invalid action')
    
    def is_goal(self, state) -> bool:
        return not any(state[1])
    
    def action_cost(self, state, action):
        return 1
    
    def heuristic(self, state):
        return sum(state[1])

## Task 1: Implement the A* algorithm

To implement the A* algorithm you must have a priority queue. Luckily, Python comes with one, so you don't need to implement it by yourself. Then, the algorithm is very simple: 
1. Start with a queue containing a single item - the initial state
2. Repeat until the queue is not empty:
  1. Pick an item with the lowest expected cost
  2. If this is the goal, return the sequence of actions necessary to reach this state
  3. Otherwise, for each available action, create a new entry in the queue corresponding to the state reached after executing the action.
  
Guard the algorithm against infinite loops: if you already visited a state, you don't need to visit it again (if your heuristic is consistent).

In the cell below implement the algorithm in a similar manner as the BFS and DFS in the previous lesson: the sole argument is an object of the class Problem and the function should return a list of actions to achieve a goal state from the initial state.
If it is impossible to reach the goal, return `None`.
Count the number of states visited during the search (e.g., the number of different states in the collection you use to guard against infinite loops) and print in out before returning from the function, it will be useful later on to compare different heuristics.

In [4]:
from queue import PriorityQueue

def astar(problem: Problem):
    q = PriorityQueue()
    curr_state = problem.initial_state
    visited = set() 
    q.put((problem.heuristic(curr_state)+0, curr_state, []))
    nr_of_it = 0
    
    while (not q.empty()):
        nr_of_it += 1
        est_cost, curr_state, list_of_steps = q.get()
        curr_cost = est_cost - problem.heuristic(curr_state)
        visited.add(curr_state)
        if (problem.is_goal(curr_state)):
            break
        for i in problem.available_actions(curr_state):
            next_state = problem.do_action(curr_state, i)
            if (next_state not in visited):
                new_list = list_of_steps + [i]
                q.put((curr_cost + problem.action_cost(next_state, i) + problem.heuristic(next_state), next_state, new_list))
    
    print ("Number of unique visited states: ", len(visited))
    if (problem.is_goal (curr_state)):
        return list_of_steps
    return (None)

Now lets test your code in the vacuum world!

In [5]:
astar(VacuumProblem())

Number of unique visited states:  4


['Suck', 'Right', 'Suck']

## Task 2: Variants of the vacuum world

Now lets consider a different take on the vacuum world in which the heuristic is not admissible and increases as the number of dirty fields decreases.

In [6]:
class VacuumProblem1(VacuumProblem):
    def action_cost(self, state, action):
        return 1
    
    def heuristic(self, state): #Nr of vertices - nr of dirty places = Nr of clean places
        return len(state[1]) - sum(state[1])
    
astar(VacuumProblem1())

Number of unique visited states:  7


['Suck', 'Right', 'Suck']

And another in which heuristic grossly overestimates the cost.

In [7]:
class VacuumProblem2(VacuumProblem):
    def action_cost(self, state, action):
        return 1
    
    def heuristic(self, state):
        return 10 * sum(state[1])
    
astar(VacuumProblem2())

Number of unique visited states:  4


['Suck', 'Right', 'Suck']

**Which of the three heuristic functions (`VacuumProblem`, `VacuumProblem1`, `VacuumProblem2`) is the best? Is it the expected answer given the properties of the heuristics? If not, explain why an unorthodox approach works better.**

The best heuristic (least visited unique states) is VacuumProblem2, which is not an expected answer (overestimating the cost).
The reason why we got this results is that in VacuumProblem we ignore spatial movement, so heuristic does not tell us anything about moves left and right we need to do, thus algorithm favourises states where no or little moves were done and VacuumProblem2 favourises moves where amount of dirt will be decreased (10 spatial moves is equal to cleaning one dirty spot) and explores them faster then VacuumProblem, which results in finding a solution faster (tested on bigger instance than (0, (True, True)), because for the instance proposed in exercise there was a draw)

## Task 3: 8-puzzle problem

Recall the 8-puzzle problem from the previous lesson. Reuse your code and implement an extended version assuming that each action costs 1. Propose 3 (at least) admissible heuristics. This time don't change the initial state, your solution should be capable enough to solve this.

![](aima-images/fig3_4.png)

In [8]:
from math import sqrt

class PuzzleProblem(Problem):
    @property
    def initial_state(self):
        return ((7, 2, 4), (5, 0, 6), (8, 3, 1))
    
    def goal_state(self):
        return ((0, 1, 2), (3, 4, 5), (6, 7, 8))
    
    def available_actions(self, state):
        return ['Left', 'Right', 'Up', 'Down']        
    
    def convert_tuple_to_list(self, state):
        y1, y2, y3 = state
        final_list = []
        for i in [y1, y2, y3]:
            x1, x2, x3 = i
            final_list.append([x1, x2, x3])
        return final_list
    
    def convert_list_to_tuple (self, state):
        y = [[], [], []]
        for i in range (3):
            x1, x2, x3 = state [i]
            y[i] = (x1, x2, x3)
        final_tuple = (y[0], y[1], y[2])
        return final_tuple
    
    def find_position (self, state, puzzle): #find position of any puzzle
        for i in range(3):
            for j in range (3):
                if (state[i][j] == puzzle):
                    return (i, j)
        print ("Sorry to say it, but puzzle evaporated ;)")
        return None
    
    def do_action(self, state, action):
        new_state = self.convert_tuple_to_list (state)
        x, y = self.find_position(new_state, 0)
        if (action == 'Up'):
            if (x > 0):
                new_state[x][y], new_state[x-1][y] = new_state[x-1][y], new_state[x][y]
            else:
                return self.convert_list_to_tuple(new_state)
        if (action == 'Down'):
            try:
                new_state[x][y], new_state[x+1][y] = new_state [x+1][y], new_state[x][y]
            except IndexError:
                return self.convert_list_to_tuple(new_state)
        if (action == 'Left'):
            if(y > 0):
                new_state[x][y], new_state[x][y-1] = new_state[x][y-1], new_state[x][y]
            else:
                return self.convert_list_to_tuple(new_state)
        if (action == 'Right'):
            try:
                new_state[x][y], new_state[x][y+1] = new_state[x][y+1], new_state[x][y]
            except IndexError:
                return self.convert_list_to_tuple(new_state)
        return self.convert_list_to_tuple(new_state)
    
    def is_goal(self, state) -> bool:
        if (state == self.goal_state()):
            return True
        return False
        
    def action_cost(self, state, action) -> float:
        return 1
        
    def heuristic(self, state) -> float:
        goal = self.convert_tuple_to_list(self.goal_state())
        heur = 0
        l = self.convert_tuple_to_list(state)
        for i in range (3):
            for j in range (3):
                a, b = self.find_position(goal, l[i][j])
                heur += sqrt(pow((a - i), 2) + pow((b - j), 2))
        return heur/2
                

**Prove that this heuristic is admissible.**

Here we calculated sum distances in a straight line between where the puzzle is and where it should be. However, when there are two puzzles that are in improper places e. g. ((1, 0, 2), (3, 4, 5), (6, 7, 8)) the heuristic would equal two and only one move is possible. That's because summing all distances is basically doubling a number of minimal moves in optimal situations (swapping one puzzle with blank space is changing position of two puzzles at once) - to correct that we need to divide our function by 2 to get rid of this doubling. Now it won't overestimate number of moves.

In [9]:
class PuzzleProblem1(PuzzleProblem):
    def heuristic(self, state) -> float:     
        goal = self.convert_tuple_to_list(self.goal_state())
        heur = 0
        l = self.convert_tuple_to_list(state)
        for i in range (3):
            for j in range (3):
                a, b = self.find_position(goal, l[i][j])
                heur += abs(a - i) + abs(b - j)
        return heur/2

**Prove that this heuristic is admissible.**

Here we calculated sum distances remembering that we can't move in a straight line. However  (as it was previously) we need to divide our sum by two, to ensure that heuristic won't be overestimated - thats why we return heur/2.

In [10]:
class PuzzleProblem2(PuzzleProblem):
    def heuristic(self, state) -> float:
        goal = self.convert_tuple_to_list(self.goal_state())
        heur = 0
        l = self.convert_tuple_to_list(state)
        for i in range (3):
            for j in range (3):
                a, b = self.find_position(goal, l[i][j])
                heur += (a != i or b != j )
        return max(0, heur-1)

**Prove that this heuristic is admissible.**

Now we calculate number of puzzles that are in the improper places. However, in each case making a move is changing a position of two puzzles at once, so to ensure that we don't overestimate we need to substract 1 because when 9 puzzles are in the wrong place, it can happen that we can achieve goal state by making 8 moves.

Run your heuristics on the given problem.

In [11]:
print("Plan 0:")
plan0 = astar(PuzzleProblem())
print(plan0, end = '\n\n')
print("Plan 1:")
plan1 = astar(PuzzleProblem1())
print (plan1, end = '\n\n')
print("Plan 2:")
plan2 = astar(PuzzleProblem2())
print(plan2, end = '\n\n')

print("Is plan0==plan1?", plan0 == plan1)
print("Is len(plan0)==len(plan1)?", len(plan0) == len(plan1))
print("Is plan0==plan2?", plan0 == plan2)
print("Is len(plan0)==len(plan2)?", len(plan0) == len(plan2))
print("Is plan1==plan2?", plan1 == plan2)
print("Is len(plan1)==len(plan2)?", len(plan1) == len(plan2))

Plan 0:
Number of unique visited states:  45532
['Left', 'Up', 'Right', 'Down', 'Right', 'Down', 'Left', 'Left', 'Up', 'Right', 'Right', 'Down', 'Left', 'Left', 'Up', 'Right', 'Right', 'Up', 'Left', 'Left', 'Down', 'Right', 'Right', 'Up', 'Left', 'Left']

Plan 1:
Number of unique visited states:  23429
['Left', 'Up', 'Right', 'Down', 'Right', 'Down', 'Left', 'Left', 'Up', 'Right', 'Right', 'Down', 'Left', 'Left', 'Up', 'Right', 'Right', 'Up', 'Left', 'Left', 'Down', 'Right', 'Right', 'Up', 'Left', 'Left']

Plan 2:
Number of unique visited states:  33086
['Left', 'Up', 'Right', 'Down', 'Right', 'Down', 'Left', 'Left', 'Up', 'Right', 'Right', 'Down', 'Left', 'Left', 'Up', 'Right', 'Right', 'Up', 'Left', 'Left', 'Down', 'Right', 'Right', 'Up', 'Left', 'Left']

Is plan0==plan1? True
Is len(plan0)==len(plan1)? True
Is plan0==plan2? True
Is len(plan0)==len(plan2)? True
Is plan1==plan2? True
Is len(plan1)==len(plan2)? True


**Which of the heuristics is the best for this task? Why is that?**

Best of presented heuristics (least visited unique states) is middle heuristic. That's because it is closest to reality - last heuristic in almost every case underestimate true cost so it is neccesary to explore more states (favourising states where few moves were done) and first heuristic returns values as it would be possible to move in a straight line (which is not possible) so it also underestimates. Middle heuristic (Manhattan Distance) is best, because even though it is not perfect, it respects the fact that blank space moves either vertically or horizontally, not diagonally.

------------
The pictures and the description of 8-puzzle are from "Artificial Intelligence: A Modern Approach" 3rd ed.